Reading instruments SDA 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime 
import os 
import glob 
import math 
import pynmea2


In [2]:
#data directory
data_dir = 'C:/Users/ica/OneDrive - Plymouth Marine Laboratory/vscode/EC_co2_flux/EC flux processing/SDA_IMC/data'

In [5]:
#get filenames in raw data files

def file_name_dir(data_dir):
    """
    get the raw EC data file names and directories

    input:  the parent directory of the raw EC data
    return: the EC data file names (_name) and directories(_dir)
    
    """    

    cr6_name = os.listdir(data_dir + '\\CR6')
    GPS_name = os.listdir(data_dir + '\\Underway')
    metek_name = os.listdir(data_dir + '\\Metek')
    cr800_name = os.listdir(data_dir + '\\CR800')
    Picarro_name = os.listdir(data_dir + '\\Picarro')

 
    return cr6_name,GPS_name,metek_name,cr800_name,Picarro_name

In [6]:
#creates lists of files to be processed 
cr6_name,GPS_name,metek_name,cr800_name,Picarro_name = file_name_dir(data_dir)

In [7]:
#make new folder to store the processed data
new_folder_path = os.path.join(data_dir, "L0_test")

# Create the new folder
os.makedirs(new_folder_path, exist_ok=True)

# subfolders to create
subfolders = ["TimeAdjGases", "PML_WindsForMotcorr", "Ship_WindsForMotcorr"]

# subfolder inside "L0"
for subfolder in subfolders:
    os.makedirs(os.path.join(new_folder_path, subfolder), exist_ok=True)


Functions to read raw data files 

In [131]:
#read underway
def read_gps_file(filepath):
    """
    Reads an NMEA file and extracts datetime, latitude, longitude, and speed.
    """
    data = {
        'datetime': [],
        'latitude': [],
        'longitude': [],
        'speed': []
    }
    
    with open(filepath) as file:
        for line in file:
            try:
                # Split timestamp from NMEA sentence
                timestamp_str, nmea = line.strip().split(' $')
                nmea = '$' + nmea
                timestamp = pd.to_datetime(timestamp_str)
                
                if "INVTG" in nmea:
                    msg = pynmea2.parse(nmea)
                    if msg.spd_over_grnd_kts:
                        speed = float(msg.spd_over_grnd_kts)
                        data['datetime'].append(timestamp)
                        data['speed'].append(speed)
                        data['latitude'].append(None)
                        data['longitude'].append(None)
                        
                elif "INGGA" in nmea:
                    msg = pynmea2.parse(nmea)
                    if msg.latitude and msg.longitude:
                        lat = round(msg.latitude,6)
                        lon = round(msg.longitude,6)
                        if msg.lat_dir == 'S':
                            lat = -lat
                        if msg.lon_dir == 'W':
                            lon = -lon
                        data['datetime'].append(timestamp)
                        data['latitude'].append(lat)
                        data['longitude'].append(lon)
                        data['speed'].append(None)
                        
            except (ValueError, pynmea2.ParseError) as e:
                continue

    df = pd.DataFrame(data)
    return df.set_index('datetime')

In [133]:
#read underway.hdg.txt
def read_hdg_file(filepath):
    data = {
        'datetime': [],
        'heading': [],
        'rate_of_turn': []
    }
    
    with open(filepath) as file:
        for line in file:
            try:
                timestamp_str, nmea = line.strip().split(' $')
                nmea = '$' + nmea
                timestamp = pd.to_datetime(timestamp_str)
                
                if "INHDT" in nmea:
                    msg = pynmea2.parse(nmea)
                    if msg.heading:
                        heading = float(msg.heading)
                        data['datetime'].append(timestamp)
                        data['heading'].append(heading)
                        data['rate_of_turn'].append(None)
                      
                elif "INROT" in nmea:
                    msg = pynmea2.parse(nmea)
                    if msg.rate_of_turn:
                        rot = float(msg.rate_of_turn)
                        data['datetime'].append(timestamp)
                        data['heading'].append(None)
                        data['rate_of_turn'].append(rot)
                
                        
            except (ValueError, pynmea2.ParseError) as e:
                continue

    
    df = pd.DataFrame(data)
    return df.set_index('datetime')

In [ ]:
nmea_hdg_df = read_hdg_file('C:/Users/ica/OneDrive - Plymouth Marine Laboratory/vscode/EC_co2_flux/EC flux processing/SDA_IMC/data/Underway/2024-06-04T18-00-00Z-hdg.txt')


In [ ]:
nmea_gps_df = read_gps_file('C:/Users/ica/OneDrive - Plymouth Marine Laboratory/vscode/EC_co2_flux/EC flux processing/SDA_IMC/data/Underway/2024-06-04T18-00-00Z-gps.txt')


In [ ]:
# Merge GPS and heading data using datetime index


Wind data 

In [135]:
for filename  in cr6_name:
    df= pd.read_csv(data_dir + '\\CR6\\' + filename, skiprows=1, delimiter = ',')
    df = df.iloc[2:].reset_index(drop=True)

In [ ]:
def process_cr6_file(df):
    # Convert TIMESTAMP to datetime
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    
    # Get the first timestamp in the data
    first_time = df['TIMESTAMP'].iloc[0]
    
    # Check if the timestamp starts at the top of the hour
    if first_time.minute != 0 or first_time.second != 0:
        # Calculate the next hour
        next_hour = first_time.replace(minute=0, second=0) + pd.Timedelta(hours=1)
        
        # Filter data to keep only rows from the next hour
        df = df[df['TIMESTAMP'] >= next_hour]
        
        # Create new filename with the next hour
        new_filename = f"CR6data_{next_hour.strftime('%Y_%m_%d_%H')}19.dat"
    else:
        new_filename = None
    
    return df, new_filename

In [137]:
def read_cr6_data(df):
    # Convert TIMESTAMP to datetime and set as index
    df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'])
    df = df.set_index('TIMESTAMP')
    
    rad2deg = 180.0 / np.pi

    # Convert columns to numeric, replacing empty strings with NaN
    numeric_cols = ['SonicX', 'SonicY', 'SonicZ', 'SonicT',
                   'ShipSonicX', 'ShipSonicY', 'ShipSonicZ', 'ShipSonicT',
                   'RotX', 'RotY', 'RotZ', 'AccX', 'AccY', 'AccZ']
    
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    # Calculate wind components
    result_df = pd.DataFrame(index=df.index)
    result_df['u_ms'] = df['SonicY'] / 100.0
    result_df['v_ms'] = df['SonicX'] / -100.0
    result_df['w_ms'] = df['SonicZ'] / 100.0
    result_df['t_degC'] = df['SonicT'] / 100.0
    
    result_df['u_ms_ship'] = df['ShipSonicY'] / 100.0
    result_df['v_ms_ship'] = df['ShipSonicX'] / -100.0
    result_df['w_ms_ship'] = df['ShipSonicZ'] / 100.0
    result_df['t_degC_ship'] = df['ShipSonicT'] / 100.0

    # Rotation and Acceleration
    result_df['rotx_degs'] = rad2deg * df['RotX'] / 1000.0
    result_df['roty_degs'] = rad2deg * df['RotY'] / 1000.0
    result_df['rotz_degs'] = rad2deg * df['RotZ'] / 1000.0
    result_df['accelx_g'] = df['AccX'] / -1000.0
    result_df['accely_g'] = df['AccY'] / -1000.0
    result_df['accelz_g'] = df['AccZ'] / -1000.0

    return result_df